#### Mutant Generation

In [5]:
import pandas as pd
import numpy as np
import spacy
import en_core_web_lg
import neuralcoref
nlp = en_core_web_lg.load()
coref = neuralcoref.NeuralCoref(nlp.vocab)
nlp.add_pipe(coref, name='neuralcoref')

In [20]:
text = "About your terrible movie copying Beethoven. As a professional musician it\'s my duty to watch every movie made about any composer and Beethoven is one of my favorites. When Hungarians and Americans meet, it\'s a terrible combination of empty over the top emotions combined with the worst taste possible. You proved it in your terrible b-movie. The only thing that carries the movie is the music. Of course you didn\'t bother to look further than the good but in my taste contrived performances of the Tackacs quartet, but OK I have to admit that the performances at least have quality as contrast to the movie you\'ve made. It starts of with the dying DEAF Beethoven who perfectly understands Anna who is merely whispering. Beethoven\'s hearing during the movie get\'s better by the minute, but that must be because of some vague divine thing. Then there is the quite impossible semi-pornographic \"eyes wide shut\" double-conducting scene which is totally over the top with the luscious Anna and the crying nephew in the end (who also cries in the deleted scenes with constant red eyes, my GOD what a performance). And as culmination the rip-off from Amadeus, with Beethoven dictating music to Anna not in notes but in total nonsense, which she understands perfectly but no-one else in your audience even trained professional musicians will understand. Of course your reaction will be that negative response is a response at least, but I can assure you that Beethoven himself is turning in his grave because of your worthless creation and with reason. This so called homage is blasphemy and I am so sorry to have rented one of the worst movies ever made even though it\'s about my favorite subject. Ed Harris and others, you cannot comprehend the greatness of Beethoven in your wildest dreams and certainly not after a couple of lessons in conducting and violin playing. That's the trouble with you Americans: you think you can grasp everything even when it takes a lifetime of hard work. Yeah we can do it anyway! Remember that a good product comes with hard labor, talent, devotion and professionalism. All these you creators of Copying Beethoven lack. See you in kindergarten."
doc = nlp(text)
sentences = [sent.string.strip() for sent in doc.sents]
len(doc)
doc[6]

.

In [6]:
class Entity:
    word = ""
    start = 0
    end = 0
    ent_type = ""
    def __init__(self, word, start, end, ent_type) :
        self.word = word
        self.start = start
        self.end = end
        self.ent_type = ent_type
    
    def __str__(self) :
        return self.word
    
    def __repr__(self) :
        return self.word
        
    def get_word(self):
        return self.word
    
    def get_start(self):
        return self.start
    
    def get_end(self):
        return self.end
    
    def get_entity_type(self):
        return self.ent_type
    
    def is_person(self):
        return self.ent_type == "PERSON"

In [7]:
# contain a word and its location inside the sentence
# The location is indicated by start char and end char
class Token: 
    word = ""
    start = 0
    end = 0
    
    def __init__(self, word, start, end) :
        self.word = word
        self.start = start
        self.end = end
        
    def __str__(self) :
        return self.word
    
    def __repr__(self) :
        return self.word
        
    def get_word(self):
        return self.word
    
    def get_start(self):
        return self.start
    
    def get_end(self):
        return self.end


# Reference is a class to save Reference data
# e.g. La Marquesa : [La Marquesa, her]
class Ref:
    
    name = ""
    reference = []
    reference_list = []
        
    def __init__(self, name, reference):
        self.name = str(name)
        self.reference = []
        self.reference_list = []
        for word in reference :
            self.reference_list.append(word.text)
            self.reference.append(Token(word.text, word.start_char, word.end_char))
            
    def __str__(self) :
        return self.name + ": " + str(self.reference_list)
    
    def __repr__(self) :
        return self.name + ": " + str(self.reference_list)
    
    def get_name(self):
        return self.name
    
    def get_reference(self):
        return self.reference
    
    def get_reference_list(self):
        return self.reference_list
    
    # is having male subject
    def is_having_male_subject(self):
        if "He" in self.reference_list :
            return True
        elif "he" in self.reference_list :
            return True
        else :
            return False

    # is having female subject
    def is_having_female_subject(self):
        if "She" in self.reference_list :
            return True
        elif "she" in self.reference_list :
            return True
        else :
            return False

#### Load gender associated word

In [8]:
# gender associated word
gaw = pd.read_csv("../data/gender_associated_word/masculine-feminine-person.txt")
gaw.head()

,masculine,feminine
0,boy,girl
1,brother,sister
2,daddy,mummy
3,man,woman
4,father,mother


#### Load Name from Gender Computer

In [9]:
gc = pd.read_csv("../data/gc_name/data.csv")
gc

,Name,Gender,Country
0,Roen,male,UK
1,Jeet,male,UK
2,Hagen,male,UK
3,Willow,male,UK
4,Belal,male,UK
...,...,...,...
305,Dódi,female,Hungary
306,Iboyka,female,Hungary
307,Zsófia,female,Hungary
308,Ágota,female,Hungary


In [10]:
gcm = gc[gc["Gender"] == "male"]
gcf = gc[gc["Gender"] == "female"]

In [12]:
# names from GC
mnames = gcm["Name"].values
fnames = gcf["Name"].values

# names from EEC paper
# mnames = ["Alonzo", "Adam", "Alphonse", "Alan", "Darnell", "Andrew", "Jamel", "Frank", "Jerome", "Harry", "Lamar", "Jack", "Leroy", "Josh", "Malik", "Justin", "Terrence", "Roger", "Torrance", "Ryan"]
# fnames = ["Ebony", "Amanda", "Jasmine", "Betsy", "Lakisha", "Courtney", "Latisha", "Ellen", "Latoya", "Heather", "Nichelle", "Katie", "Shaniqua", "Kristin", "Shereen", "Melanie", "Tanisha", "Nancy", "Tia", "Stephanie"]

# small name for debugging
mnames = ["Alonzo"] 
fnames = ["Ebony"]

# masculine pronoun
masculine_pronoun = ["he", "him", "his", "himself", "He", "Him", "His", "Himself"]

# feminine prononun
feminine_pronoun = ["she","her", "her", "herself", "She","Her", "Her", "Herself"]

# gender flipper
masculine_flipper = {}
feminine_flipper = {}

for _m, _f in zip(masculine_pronoun, feminine_pronoun) :
    feminine_flipper[_m] = _f
    masculine_flipper[_f] = _m

In [27]:
class Coref:
    original = ""
    resolved = ""
    refs = []
    person_entities = []
    is_one_subject = False
    is_male = False
    chunk = []
    person_reference = None
    person_name = None
    person_substitution = None
    
    def __init__(self, text):
        
        self.original = str(text)
        doc = nlp(text)
        
        self.person_entities = self.get_person_entities(doc.ents)
        
        self.resolved = str(doc._.coref_resolved)
        self.refs = []
        refs = doc._.coref_clusters
        for r in refs :
            self.refs.append(Ref(r.main, r.mentions))
        
        self.is_one_subject, self.person_reference, self.is_male = self.check_one_subject()
        
        if self.is_one_subject :
            self.chunk = self.generate_chunk_from_coref()
            
    def get_original(self):
        return self.original
    
    def get_resolved(self):
        return self.resolved
    
    def get_refs(self):
        return self.refs
    
    def get_gender(self):
        if self.is_male :
            return "male"
        return "female"
    
    def get_person_entities(self, ents) :
        entities = set()
        for ent in ents :
            e = Entity(ent.text, ent.start_char, ent.end_char, ent.label_)
            if e.is_person() :
                entities.add(e.get_word())
        return list(entities)
    
    def is_having_one_subject(self) :
        return self.is_one_subject
    
    def is_the_person_reference_is_a_person_name(self) :
        if self.person_reference.get_name() in self.person_entities :
            return True
        return False
    
    def is_the_person_reference_is_followed_by_an_aposthrope(self) :
        if self.person_reference.get_name()[-2:] == "'s" :
            return True
        return False
    
    def is_the_person_name_only_occupies_several_words_in_the_person_reference(self) :
        doc_person_reference = nlp(self.person_reference.get_name())

        for token in doc_person_reference:
#             print(token.text, token.pos_, token.dep_)
            if token.text in self.person_entities and token.dep_ == "ROOT":        
                return True
        return False
        
    def is_the_main_person_reference_is_a_pronoun(self):
        return self.person_reference.get_name() in (masculine_pronoun + feminine_pronoun)
    
    def is_there_is_a_person_name_inside_the_references(self) :
        for token in self.person_reference.get_reference_list() :
            if token in self.person_entities :
                return True
        return False
    
    def is_the_person_reference_is_a_gender_associated_word(self) :
        
        doc_person_reference = nlp(self.person_reference.get_name())

        main_token = None
        for token in doc_person_reference:
#             print(token.text, token.pos_, token.dep_)
            if token.pos_ == "NOUN" and token.dep_ == "ROOT" :
                main_token = token.text
        
        if main_token != None :
            if self.is_male :
                if main_token in gaw["masculine"].values :
                    return True
            else :
                if main_token in gaw["feminine"].values :
                    return True
            
        return False
        
        
    def check_one_subject(self) :
                
        s = 0
        subject_reference = None
        for r in self.refs :
            if r.is_having_male_subject() :
                s += 1
                subject_reference = r
                is_male = True
            
            if r.is_having_female_subject() :
                s += 1
                subject_reference = r
                is_male = False
                
        if s == 1 :
            # check if it's only prononun there
            is_only_pronoun = True
            for r in subject_reference.get_reference() :
                if r.word not in masculine_pronoun and r.word not in feminine_pronoun :
                    is_only_pronoun = False

            if is_only_pronoun :
                return False, None, None 
            
            return True, subject_reference, is_male
        else :
            return False, None, None
    
    def get_person_reference(self):
        return self.person_reference
    
    def generate_chunk_from_coref(self) :
        chunk = []
        refs = self.person_reference.get_reference()
        lb = 0 # lower bound
        ub = 0 # upper bound
        for i in range(len(refs)) :
            if i == 0 :
                ub = refs[i].start
                _chunk = text[lb:ub]
                if _chunk == "" :
                    chunk.append(" ")
                else :
                    chunk.append(_chunk)
            else :
                lb = refs[i-1].end
                ub = refs[i].start
                _chunk = text[lb:ub]
                if _chunk == "" :
                    chunk.append(" ")
                else :
                    chunk.append(_chunk)
                
            if i == len(refs)-1 :
                lb = refs[-1].end
                chunk.append(self.original[lb:])
        
        return chunk
    
    def generate_normal_male_mutant_text(self):
        refs = self.person_reference.get_reference()
        chunk = self.chunk
        mutant = []
        if self.is_male :
            for name in mnames :
                t = []
                t.append(chunk[0])
                i = 1
                for r in refs :
                    if r.word in masculine_pronoun :
                        t.append(r.word)
                    else :
                        t.append(name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "male", "text": "".join(t), "mutant_generation": "normal", "person_reference": name})
        else :
            gender = "female"
            for name in mnames :
                t = []
                t.append(chunk[0])
                i = 1
                for r in refs :
                    if r.word in feminine_pronoun :
                        t.append(masculine_flipper[r.word])
                    else :
                        t.append(name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "male", "text": "".join(t), "mutant_generation": "normal", "person_reference": name})
        return mutant
            
    def generate_normal_female_mutant_text(self):
        refs = self.person_reference.get_reference()
        chunk = self.chunk
        mutant = []
        if self.is_male :
            for name in fnames :
                t = []
                t.append(chunk[0])
                i = 1
                for r in refs :
                    if r.word in masculine_pronoun :
                        t.append(feminine_flipper[r.word])
                    else :
                        t.append(name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "female", "text": "".join(t), "mutant_generation": "normal", "person_reference": name})
        else :
            for name in fnames :
                t = []
                t.append(chunk[0])
                i = 1
                for r in refs :
                    if r.word in feminine_pronoun :
                        t.append(r.word)
                    else :
                        t.append(name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "female", "text": "".join(t), "mutant_generation": "normal", "person_reference": name})
        return mutant
    
    def generate_apostrophe_male_mutant_text(self):
        refs = self.person_reference.get_reference()
        chunk = self.chunk
        mutant = []
        if self.is_male :
            for name in mnames :
                t = []
                t.append(chunk[0])
                i = 1
                for r in refs :
                    if r.word in masculine_pronoun :
                        t.append(r.word)
                    elif r.word[-2:] == "'s" :
                        t.append(name + "'s")
                    else :
                        t.append(name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "male", "text": "".join(t), "mutant_generation": "apostrophe", "person_reference": name + "'s"})
        else :
            gender = "female"
            for name in mnames :
                t = []
                t.append(chunk[0])
                i = 1
                for r in refs :
                    if r.word in feminine_pronoun :
                        t.append(masculine_flipper[r.word])
                    elif r.word[-2:] == "'s" :
                        t.append(name + "'s")
                    else :
                        t.append(name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "male", "text": "".join(t), "mutant_generation": "apostrophe", "person_reference": name + "'s"})
        return mutant
    
                
    def generate_apostrophe_female_mutant_text(self):
        refs = self.person_reference.get_reference()
        chunk = self.chunk
        mutant = []
        if self.is_male :
            for name in fnames :
                t = []
                t.append(chunk[0])
                i = 1
                for r in refs :
                    if r.word in masculine_pronoun :
                        t.append(feminine_flipper[r.word])
                    elif r.word[-2:] == "'s" :
                        t.append(name + "'s")
                    else :
                        t.append(name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "female", "text": "".join(t), "mutant_generation": "apostrophe", "person_reference": name + "'s"})
        else :
            for name in fnames :
                t = []
                t.append(chunk[0])
                i = 1
                for r in refs :
                    if r.word in feminine_pronoun :
                        t.append(r.word)
                    elif r.word[-2:] == "'s" :
                        t.append(name + "'s")
                    else :
                        t.append(name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "female", "text": "".join(t), "mutant_generation": "apostrophe", "person_reference": name + "'s"})
        return mutant
    
    def get_person_substitution(self) :
                
        doc_person_reference = nlp(self.person_reference.get_name())

        person_substitution = ""
        
        start_marker = False
        
        for token in doc_person_reference:
            
            if start_marker == True :
                person_substitution += (token.text + " ")
            else :
                if token.text in self.person_entities and token.dep_ == "ROOT":
                    start_marker = True
                    person_substitution += "@ROOT "
            
        person_substitution = person_substitution[:-1] 
        
        return person_substitution
    
    def generate_male_mutant_text_by_specific_person_reference_template(self) :
        person_substitution = self.get_person_substitution()
        
        refs = self.person_reference.get_reference()
        chunk = self.chunk
        mutant = []
        if self.is_male :
            for name in mnames :
                t = []
                t.append(chunk[0])
                i = 1
                substituted_name = person_substitution.replace("@ROOT", name)
                for r in refs :
                    if r.word in masculine_pronoun :
                        t.append(r.word)
                    else :
                        t.append(substituted_name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "male", "text": "".join(t), "mutant_generation": "part of person reference", "person_reference": substituted_name })
        else :
            gender = "female"
            for name in mnames :
                t = []
                t.append(chunk[0])
                i = 1
                substituted_name = person_substitution.replace("@ROOT", name)
                for r in refs :
                    if r.word in feminine_pronoun :
                        t.append(masculine_flipper[r.word])
                    else :
                        t.append(substituted_name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "male", "text": "".join(t), "mutant_generation": "part of person reference", "person_reference": substituted_name})
        return mutant
    
        
    def generate_female_mutant_text_by_specific_person_reference_template(self) :
        person_substitution = self.get_person_substitution()
        
        refs = self.person_reference.get_reference()
        chunk = self.chunk
        mutant = []
        
        if self.is_male :
            for name in fnames :
                t = []
                t.append(chunk[0])
                i = 1
                substituted_name = person_substitution.replace("@ROOT", name)
                for r in refs :
                    if r.word in masculine_pronoun :
                        t.append(feminine_flipper[r.word])
                    else :
                        t.append(substituted_name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "female", "text": "".join(t), "mutant_generation": "part of person reference", "person_reference": substituted_name})
        else :
            for name in fnames :
                t = []
                t.append(chunk[0])
                i = 1
                substituted_name = person_substitution.replace("@ROOT", name)
                for r in refs :
                    if r.word in feminine_pronoun :
                        t.append(r.word)
                    else :
                        t.append(substituted_name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "female", "text": "".join(t), "mutant_generation": "part of person reference", "person_reference": substituted_name})
        return mutant
    
    def get_gender_associated_word_substitution(self) :
                
        doc_person_reference = nlp(self.person_reference.get_name())

        word_substitution = ""
        
        for token in doc_person_reference:
            
            if token.dep_ == "ROOT":
                start_marker = True
                word_substitution += "@ROOT "
            else :
                word_substitution += (token.text + " ")

        word_substitution = word_substitution[:-1] 
        
        return word_substitution
    
    def generate_male_mutant_using_gender_associated_word(self) :
        substituted_word = self.get_gender_associated_word_substitution()
        
        refs = self.person_reference.get_reference()
        chunk = self.chunk
        mutant = []
        if self.is_male :
            for name in gaw["masculine"].values :
                t = []
                t.append(chunk[0])
                i = 1
                substituted_name = substituted_word.replace("@ROOT", name)
                for r in refs :
                    if r.word in masculine_pronoun :
                        t.append(r.word)
                    else :
                        t.append(substituted_name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "male", "text": "".join(t), "mutant_generation": "gender associated word", "person_reference": substituted_name})
        else :
            gender = "female"
            for name in gaw["masculine"].values :
                t = []
                t.append(chunk[0])
                i = 1
                substituted_name = substituted_word.replace("@ROOT", name)
                for r in refs :
                    if r.word in feminine_pronoun :
                        t.append(masculine_flipper[r.word])
                    else :
                        t.append(substituted_name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "male", "text": "".join(t), "mutant_generation": "gender associated word", "person_reference": substituted_name})
        return mutant
        
    def generate_female_mutant_using_gender_associated_word(self) :
        substituted_word = self.get_gender_associated_word_substitution()
        
        refs = self.person_reference.get_reference()
        chunk = self.chunk
        mutant = []
    
        if self.is_male :
            for name in gaw["feminine"].values :
                t = []
                t.append(chunk[0])
                i = 1
                substituted_name = substituted_word.replace("@ROOT", name)
                for r in refs :
                    if r.word in masculine_pronoun :
                        t.append(feminine_flipper[r.word])
                    else :
                        t.append(substituted_name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "female", "text": "".join(t), "mutant_generation": "gender associated word", "person_reference": substituted_name})
        else :
            for name in gaw["feminine"].values :
                t = []
                t.append(chunk[0])
                i = 1
                substituted_name = substituted_word.replace("@ROOT", name)
                for r in refs :
                    if r.word in feminine_pronoun :
                        t.append(r.word)
                    else :
                        t.append(substituted_name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "female", "text": "".join(t), "mutant_generation": "gender associated word", "person_reference": substituted_name})
        return mutant

        
    def generate_normal_mutant_text(self):
        chunk = self.chunk
        mutants = []
        male_mutant = self.generate_normal_male_mutant_text()
        for _mutant in male_mutant :
            mutants.append(_mutant)
        female_mutant = self.generate_normal_female_mutant_text()
        for _mutant in female_mutant :
            mutants.append(_mutant)
        return mutants
        
    def generate_apostrophe_mutant_text(self):
        chunk = self.chunk
        mutants = []
        male_mutant = self.generate_apostrophe_male_mutant_text()
        for _mutant in male_mutant :
            mutants.append(_mutant)
        female_mutant = self.generate_apostrophe_female_mutant_text()
        for _mutant in female_mutant :
            mutants.append(_mutant)
        return mutants

    def generate_mutant_text_from_the_part_of_person_reference(self) :
        chunk = self.chunk
        mutants = []
        male_mutant = self.generate_male_mutant_text_by_specific_person_reference_template()
        for _mutant in male_mutant :
            mutants.append(_mutant)
        female_mutant = self.generate_female_mutant_text_by_specific_person_reference_template()
        for _mutant in female_mutant :
            mutants.append(_mutant)
        return mutants
    
    def generate_mutant_using_gender_associated_word(self) :
        chunk = self.chunk
        mutants = []
        male_mutant = self.generate_male_mutant_using_gender_associated_word()
        for _mutant in male_mutant :
            mutants.append(_mutant)
        female_mutant = self.generate_female_mutant_using_gender_associated_word()
        for _mutant in female_mutant :
            mutants.append(_mutant)
        return mutants
    
    def generate_mutants(self) :
        mutants = []
        if c.is_the_person_reference_is_a_person_name() :
#             print("The person reference is a person name")
            if (c.is_the_person_reference_is_followed_by_an_aposthrope()) :
#                 print("it's followed by an aposthrope")
                mutants = c.generate_apostrophe_mutant_text()
            else :
#                 print("it's not followed by an aposthrope")
                mutants = c.generate_normal_mutant_text()
        elif c.is_the_person_name_only_occupies_several_words_in_the_person_reference() :
#             print("The person name only occupies several words in the person reference")
            mutants = c.generate_mutant_text_from_the_part_of_person_reference()
        elif c.is_the_main_person_reference_is_a_pronoun() :
#             print("The main person reference is a pronoun")
            if c.is_there_is_a_person_name_inside_the_references() :
#                 print("There is a person name inside the references")
                mutants = c.generate_normal_mutant_text()
            else :
                mutants = []
#                 print("There isn't any person name inside the references")
        elif c.is_the_person_reference_is_a_gender_associated_word() :
#             print("The person reference is a gender associated word")
            mutants = c.generate_mutant_using_gender_associated_word()
        else :
            mutants = []
#             print("Skip")
        
        return mutants

In [38]:
# text = "The pace of the film is ponderously slow in parts, but if you can tune into its languid speed and lengthy silences then it is a satisfying piece of courtly intrigue. The story of the first Emperor of China, his childhood sweetheart and the personal cost of power. The film is very atmospheric, the extremely mannered and polite courtly ceremony and ritual contrasted with sudden brutal violence. Filmed in a way that evokes shadows and cold spaces. Battle scenes are rare and short, the focus is on the battle within the individual on what is right to do and whether the ends justify the means. The emperor's journey from idealistic peacemaker to ruthless tyrant is aiming to be subtle, but gives little background or convincing insight into the motivation of the Emperor, indeed his actions and aims do not really change throughout, only Gong Li's attitudes to him are altered. The most interesting performances are Gong Li's and the titular Assassin as they reassess when to fight, when to retreat, when to kill. The most expensive film ever made in China at the time, the Emperor and the Assassin does not rely on hysteric emotion or big battles, but rather a brooding atmosphere of menace and inevitability. Gong Li fans will be unsurprised to hear she is as stunningly beautiful as ever, giving an understated performance."
# text = "Of all of the post-1985 Perry Mason movies I have seen, this one is my least favorite. I confess I have never liked Diana Muldaur as an actress. She only seems to know how to play one type of character - a hard-bitten career woman with some undefined chip on her shoulder who for that reason is extremely difficult to in any way sympathize with. This one is no exception - it runs true to form. The only thing that saves this movie, in my opinion, is an earnest performance by Scott Baio as the prosecutor - I actually found myself rooting for him to win, and the movie is worth seeing for him alone."
# text = "When Nathaniel Kahn embarked into this voyage, he hardly knew who his father really was. By the end of the film, he found him and comes to terms with the strange life he lived as a child. Louis Kahn was the father. He was an architect's architect. His designs were perhaps too complex, as he tried to create buildings that didn't conform with trends popular at that time. It is ironic that he never achieved the fame that came so easy to some of his contemporaries. He had a vision and he never strayed from it. We can see characteristics of his unique style in the buildings he left behind as a legacy to humanity. Every one of his creations are unique in that they don't imitate works from other architects. Louis Kahn's life was rather complicated. He was married, yet he had affairs with two of his assistants that produced a girl and a boy, besides the legitimate daughter he had with his wife. As a boy, Nathaniel Kahn's life was lived in a secluded area, away from his father, who only visited late at night. Louis Kahn never recognized these children, although it is very clear they all knew about the others existence.  It is tragic that Louis Kahn died alone in Grand Central Station when he was returning from a trip without making peace with the women and children he never acknowledged as his own by his side. He probably cared a great deal about all his children, but he remains an aloof figure throughout the film. We never get to know the man, although at the end, Nathaniel, in his quest to discover his father's life, finds most of the missing pieces of the puzzle. This is a personal account on the life of an artist. Thanks to that son, who has the courage to tell the story, we are almost prying into the lives of Louis Kahn and his extended family."
# text = "Ok, so I saw this movie at this year's Sundance, and I was sorely unimpressed. It took a good fifteen minutes of footage before there was an edit or a line of dialogue that made any sense, and it took another 30 minutes before the ham-fisted script gave way to a working plot that wasn't contingent on a close-up of Ryan Gosling's smile or contrived moralizing. After the first 45 minutes however, the script blossomed into a watch-able albeit not completely entertaining or thought-provoking. The highlights certainly include both Gosling and Morse's acting, Gosling being an up-and-coming star, and Morse being an extremely well-established character actor with a good feel for disparate emotions. As a sidenote, after the screening I was talking a little smack about the movie to some of my friends when David Morse walked right behind me--He looks like the nicest guy in the world, but he's a solid 6'2\" and probably outweighs me by 50 pounds. I removed my foot from my mouth and promptly changed the subject."
# text = "Bad news for anyone wanting to film a full-length parody of a Lifetime Network movie- the makers of A Deadly Encounter have already done it, albeit unintentionally. All of the Lifetime tropes are there- a divorced mother in peril from a deranged stalker, an unreliable ex-husband (who, of course, cheated on her while they were married), and a police department that patronizingly dismisses her complaints, forcing her to Stand Up For Herself. Especially jaw-dropping is the scene where the heroine, after enduring a break-in and the attempted murder of her mother by her seemingly ubiquitous stalker, decides to relieve the stress by going shopping! Having seen first hand the harassment of a co-worker by some creep she met at church, I know stalking is not a joke, but it certainly is in this movie."
# text = "Sorry folks, I love Ray Bolger's work but the one thing he ain't is a leading man. Maybe if you pretend he's the last man on earth, this romantic plot might work but come'on now ! Here's a movie that exists simply to showcase the title song which was a big hit for the Basie Band the year before (1951). And some pretty nifty singing and dancing save it from being a total disaster.  However, the story line is pathetic, even by 1952 musical comedy standards. And the other songs are equally as forgettable as Evening In Paris cologne. The dialogue embarrasses the stars, Day & Bolger. Only Claude Dauphin's Boyeresque charms keep his character three dimensional. So, how to enjoy this movie on video ?  A.) Fast forward through all the dialogue... B.) Surrender yourself to Doris Day's vocals and Ray Bolger's loose-limbed footwork. And don't miss Dauphin's hilarious take on a rain-soaked, windswept reprise of \"April In Paris\"... C.) Finally, keep a couple of bottles of Cabernet chilled and handy. Bob Raymond"
# text = "Meek and mild Edward G. Robinson (as Wilbert Winkle) decides to quit his bank job and do what he wants, open a \"fix-it\" repair shop behind his house. Mr. Robinson is married, but childless; he has befriended local orphanage resident Ted Donaldson (as Barry). Young Donaldson is an eager workshop assistant, and sees Robinson as a father figure. Robinson's nagging wife Ruth Warwick (as Amy) is unhappy with Robinson's job choice, and conspires to return matters to her idea of normalcy. Their lives are further disrupted when Robinson is drafted. The war also disrupts what might have been an interesting story, as Robinson's character struggles against a domineering, unsympathetic wife. Possibly, filmmakers are showing how war can save marriages and positively redirect lives. Robinson and Donaldson are a likable team. Robert Mitchum has an inauspicious bit part. The ending \"trick\" played by Ms. Warwick and Donaldson is predictably staged."
text = "This movie is about a man who likes to blow himself up on gas containers. He also loves his mommy. So, to keep the money coming in, he takes his act to Broadway. SEE! CODY POWERS JARRETT BLOW HIMSELF UP ON HIS BIGGEST GAS CONTAINER YET! TONIGHT! 7.30PM!  However, one day, his mommy dies and Jarrett goes berserk. He kidnaps the audience in the theatre and makes them all stand on top of a huge gas cylinder. Losing control further, he makes them all scream \"MADE IT MA, TOP OF THE CYLINDER!\" in unison. The noise is so deafening that it bursts Jarrets eardrums, causing him to topple from the cylinder into a vat of acid. This Warner Bros. movie is not all it's cracked up to be."
# text = "This is a brilliant film. The story starts off in Romania,where Billy Crystal stars as an agent who can talk anybody into doing anything. There is a really funny scene here, where Crystal is a child, and talks his father, who is a rabbi into eating pork (you filmophiles may know that this is a parody of Crystal's religion). He is mainly a talent scout, and he's trying to find a suitable villain for his latest movie. After finding out that the whole film has a terrible crew, he accidently falls into a small river. Believing he is dead, some huge hands come out to reach him. HE later thinks he is in heaven. He finds out that he isn't, but that he is in a monastery. Believing God saved him, he becomes a monk, searching for who saved him. He later finds a 7 ft 7 in man, who he casts as the perfect villain (he also recites Shakespeare!). See the film and find out the rest!"
# text = "Terror in the Jungle is a real find. If you saw it, you're one of the few lucky ones. It's hilarious! The story is about an airplane crashing in the middle of the south american jungle. The crash scene has to be seen to be believed. Everyone dies in the crash or they're subsequently eaten alive by crocodiles. Only a young blonde boy survives. A nearby tribes brings the kid to their village and they (all males) venerate him because of his golden hair! I kid you NOT! At the end, there's a lot of wrestling between the natives and the man on the search for any survivors of the downed airplane. All the while, the kid sits on a throne and his blond hair is surrounded by a golden halo and he cries nonstop!! It's a hoot!!! Very obscure and contains very questionable subtexts. A must if you're into obscure, it's-so-bad-it's-good movies."
# text = "Even the manic loony who hangs out with the bad guys in \"Mad Max\" is there. That guy from \"Blade Runner\" also cops a good billing, although he only turns up at the beginning and the end of the movie."
text

'This movie is about a man who likes to blow himself up on gas containers. He also loves his mommy. So, to keep the money coming in, he takes his act to Broadway. SEE! CODY POWERS JARRETT BLOW HIMSELF UP ON HIS BIGGEST GAS CONTAINER YET! TONIGHT! 7.30PM!  However, one day, his mommy dies and Jarrett goes berserk. He kidnaps the audience in the theatre and makes them all stand on top of a huge gas cylinder. Losing control further, he makes them all scream "MADE IT MA, TOP OF THE CYLINDER!" in unison. The noise is so deafening that it bursts Jarrets eardrums, causing him to topple from the cylinder into a vat of acid. This Warner Bros. movie is not all it\'s cracked up to be.'

In [39]:
c = Coref(text)

In [40]:
c.generate_mutants()

[{'gender': 'male',
  'text': 'This movie is about a man who likes to blow himself up on gas containers. He also loves his mommy. So, to keep the money coming in, he takes his act to Broadway. SEE! CODY POWERS Alonzo BLOW Alonzo UP ON Alonzo BIGGEST GAS CONTAINER YET! TONIGHT! 7.30PM!  However, one day, his mommy dies and Alonzo goes berserk. He kidnaps the audience in the theatre and makes them all stand on top of a huge gas cylinder. Losing control further, he makes them all scream "MADE IT MA, TOP OF THE CYLINDER!" in unison. The noise is so deafening that it bursts Jarrets eardrums, causing him to topple from the cylinder into a vat of acid. This Warner Bros. movie is not all it\'s cracked up to be.',
  'mutant_generation': 'normal',
  'person_reference': 'Alonzo'},
 {'gender': 'female',
  'text': 'This movie is about a man who likes to blow herself up on gas containers. She also loves her mommy. So, to keep the money coming in, she takes her act to Broadway. SEE! CODY POWERS Ebony